In [1]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv
from openai import OpenAI
import pandas as pd

# 環境変数の取得
load_dotenv()

# OpenAI APIクライアントを生成
client = OpenAI(api_key=os.environ['API_KEY'])

# モデル名
MODEL_NAME = "gpt-4o-mini"

In [2]:
# 1. Excelファイルを読み込む
df = pd.read_excel('サンプルデータ.xlsx', sheet_name='売上データ')
# データフレームを表示して確認
df.head()

,カテゴリー,商品コード,商品名,売上日,単価,数量,原価
0,食品,1001,りんご,2023-01-01,200,50,120
1,食品,1002,バナナ,2023-01-01,150,100,80
2,食品,1003,牛乳,2023-01-02,180,80,100
3,衣服,2001,Tシャツ,2023-01-02,1500,20,800
4,衣服,2002,ジーンズ,2023-01-03,5000,10,2500


In [3]:
sales_data_text = df.astype(str)
prompt_text = f"売上データ\n{sales_data_text}\nこの売上データの傾向を分析してください。"
print(prompt_text)

売上データ
    カテゴリー 商品コード      商品名         売上日    単価   数量    原価
0      食品  1001      りんご  2023-01-01   200   50   120
1      食品  1002      バナナ  2023-01-01   150  100    80
2      食品  1003       牛乳  2023-01-02   180   80   100
3      衣服  2001     Tシャツ  2023-01-02  1500   20   800
4      衣服  2002     ジーンズ  2023-01-03  5000   10  2500
..    ...   ...      ...         ...   ...  ...   ...
235    衣服  2077   レインパンツ  2023-04-28  2000   18  1000
236    食品  1085      ザクロ  2023-04-29   600   40   300
237   日用品  3077    バスブラシ  2023-04-29   400   60   200
238    衣服  2078  レインシューズ  2023-04-30  2500   15  1250
239    食品  1086    ココナッツ  2023-04-30   300   80   150

[240 rows x 7 columns]
この売上データの傾向を分析してください。


In [4]:
# 3. OpenAI APIの呼び出し

# 役割を設定
role = "あなたはマーケティング分野に精通したデータサイエンティストです。企業の成長をサポートするために、効果的なインサイトを提供します。"

# APIへリクエスト
response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "system", "content": role},
        {"role": "user", "content": prompt_text},
    ],
)

# LLMからの回答を表示
print(response.choices[0].message.content.strip())

この売上データに基づいて、いくつかの重要な傾向やインサイトを分析できます。以下はその過程と結論の概略です。

1. **売上カテゴリー分析**:
   - データには「食品」、「衣服」、「日用品」の3つのカテゴリーがあります。それぞれの売上人数および総売上額を集計し、カテゴリー間の売上バランスや比率を分析します。
   - どのカテゴリーが最も売上につながっているか、または期待したほど売上が伸びていないかを特定します。

2. **売上日別分析**:
   - 日別の売上推移をグラフ化することで、特定の期間（例: 祝日やプロモーション期間）における売上の増加を視覚的に捉えることができます。ピーク日や売上が低迷している日を特定し、それに基づくマーケティング戦略を考案します。

3. **商品別分析**:
   - 各商品の販売数量や売上を分析し、最も人気のある商品や売れ行きが悪い商品を特定します。これにより、売れ筋商品をさらに強化したり、未販売商品へのプロモーションを考える手がかりが得られます。

4. **利益の分析**:
   - 売上と原価を基に、各商品の粗利（単価 - 原価）を計算します。そして、商品ごと、カテゴリーごとの利益を分析します。
   - 利益率が高い商品やカテゴリー、逆にコストがかかりすぎている商品について評価することで、商品の価格戦略やコスト削減の可能性を探ります。

5. **季節性分析**:
   - 売上データは時間の経過に従って変動するため、特定の季節や月による影響も考慮する必要があります。特に食品や衣服は季節によって需要が変わる場合があります。

6. **プロモーション効果の測定**:
   - 過去のプロモーションやセールのデータがある場合、プロモーションの実施前後での売上の変化を比較し、プロモーションの効果を測定します。

### 結論
これらの分析をもとに、企業は以下のような戦略を立てることができます。

- 高売上かつ高利益の商品をプロモーションの中心に置く。
- 売上が低迷している商品には割引キャンペーンや新しいマーケティング戦略を適用する。
- 売上のピークがある日に合わせて新商品の投入を検討する。
- 季節に応じた商品ラインナップの見直しを行う。

このような分析を通じて、企業の成長戦略や販売戦略をより効果的に策定する

In [5]:
# 4. 分析結果をデータフレームに変換
result_list = response.choices[0].message.content.strip().split("\n")
df_out = pd.DataFrame(result_list, columns=['結果'])
print(df_out)

                                                   結果
0   この売上データに基づいて、いくつかの重要な傾向やインサイトを分析できます。以下はその過程と結...
1                                                    
2                                   1. **売上カテゴリー分析**:
3      - データには「食品」、「衣服」、「日用品」の3つのカテゴリーがあります。それぞれの売...
4      - どのカテゴリーが最も売上につながっているか、または期待したほど売上が伸びていないか...
5                                                    
6                                      2. **売上日別分析**:
7      - 日別の売上推移をグラフ化することで、特定の期間（例: 祝日やプロモーション期間）に...
8                                                    
9                                       3. **商品別分析**:
10     - 各商品の販売数量や売上を分析し、最も人気のある商品や売れ行きが悪い商品を特定します...
11                                                   
12                                      4. **利益の分析**:
13     - 売上と原価を基に、各商品の粗利（単価 - 原価）を計算します。そして、商品ごと、カ...
14     - 利益率が高い商品やカテゴリー、逆にコストがかかりすぎている商品について評価すること...
15                                                   
16                                      5. **季節性分析**:
17     - 売上データは時間の経過に従って変動する

In [6]:
# 5. 結果をExcelファイルに保存
df_out.to_excel("売上データ分析結果.xlsx", index=False)

In [7]:
# ワークフロー化
print("処理を開始します。")

# 1. Excelファイルを読み込む
df = pd.read_excel('サンプルデータ.xlsx', sheet_name='売上データ')

# 2. データをLLM用にテキスト形式に変換
sales_data_text = df.astype(str)
prompt_text = f"売上データ:\n{sales_data_text}\nこの売上データの傾向を分析してください。"

# 3. OpenAI APIの呼び出し
# 役割を設定
role = "あなたはマーケティング分野に精通したデータサイエンティストです。企業の成長をサポートするために、効果的なインサイトを提供します。"
# APIへリクエスト
response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "system", "content": role},
        {"role": "user", "content": prompt_text},
    ],
)

# 4. 分析結果をデータフレームに変換
result_list = response.choices[0].message.content.strip().split("\n")
df_out = pd.DataFrame(result_list, columns=['結果'])

# 5. 結果をExcelファイルに保存
df_out.to_excel("売上データ分析結果.xlsx", index=False)

print("Excelファイルに分析結果を保存しました。")

処理を開始します。
Excelファイルに分析結果を保存しました。
